### Required Libraries

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


### Load Dataset

In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
model_data.shape

(478328, 2831)

In [4]:
# check unqiue latest feature id
model_data['latest_feature_id'].nunique()

478328

In [5]:
model_data['unique_reference_number'].nunique()

478327

In [6]:
# Drop Null values of Target 
df = model_data.dropna(subset=['90_in_24mob'])
# take one copy for model development data
data = df.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

### Split the Time interval

In [7]:
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]

In [8]:
model_df1.shape, oot_df1.shape

((39010, 2831), (12128, 2831))

In [9]:
features_list =[
           
'distinct_merchant',
'SUM(raw.amount WHERE action = 1)/SUM(raw.amount)#ever',
'MEAN(raw.amount WHERE device_os_category = Windows)#ever',
'NUM_UNIQUE(pg_success_raw_data.month_end_start_category)',
 'ENTROPY(raw.mode_cleaned)#720',
'max_one_time_spent',
'amt_first_android_txn',
'MAX(pg_success_raw_data.amount)',
'top_payment_mode_count',
'amt_first_cc_txn',
'STD(pg_success_raw_data.amount)',
'month_year',
'90_in_24mob'       
 ]




In [10]:
# Selected Features for model build purpose
model_df = model_df1[features_list]

In [11]:
oot_df = oot_df1[features_list]

In [12]:
model_df.shape, oot_df.shape

((39010, 13), (12128, 13))

In [13]:
model_df['month_year'].isnull().sum()

np.int64(0)

In [14]:
#Handling Null Values
for i in model_df.columns:
    model_df[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3025534/2889689820.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  model_df[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3025534/2889689820.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df[i].fillna(-9999,inplace=True)


In [15]:
#Handling Null Values
for i in oot_df.columns:
    oot_df[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3025534/1074864086.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  oot_df[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3025534/1074864086.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oot_df[i].fillna(-9999,inplace=True)


In [16]:
model_df['90_in_24mob'].value_counts(dropna=False)

90_in_24mob
0.0    32389
1.0     6621
Name: count, dtype: int64

In [17]:
oot_df['90_in_24mob'].value_counts(dropna=False)

90_in_24mob
0.0    10039
1.0     2089
Name: count, dtype: int64

In [18]:
#take out record only present in pg data(exclude null records)
model_df = model_df[(model_df !=-9999).all(axis=1)]

In [19]:
model_df.shape

(14512, 13)

In [20]:
oot_df = oot_df[(oot_df !=-9999).all(axis=1)]

In [21]:
oot_df.shape

(4399, 13)

In [22]:
# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '90_in_24mob']) # Adjust 'target' to your actual target column name
y_model = model_df['90_in_24mob']

In [23]:
X_oot = oot_df.drop(columns=['month_year', '90_in_24mob'])
y_oot = oot_df['90_in_24mob']

In [33]:
X_oot.shape,y_oot.shape

((4399, 11), (4399,))

In [24]:
X_model.shape

(14512, 11)

### Split Model dataset into Train and Test

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [26]:
X_train.shape, X_test.shape

((8707, 11), (5805, 11))

In [27]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(6965, 11) (1742, 11) (5805, 11)


In [28]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [29]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
     'max_depth': 1,
    'subsample': 0.6,
    'colsample_bytree': 0.6,
    'lambda': 7,
    'alpha':5,
    'gamma':5,
   
     # 'scale_pos_weight':3.2,
     'random_state': 42
    
}

evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 40,
    early_stopping_rounds= 40,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.52722	eval-auc:0.51442
[10]	train-auc:0.56342	eval-auc:0.53390
[20]	train-auc:0.56748	eval-auc:0.53368
[30]	train-auc:0.57068	eval-auc:0.53765
[39]	train-auc:0.57056	eval-auc:0.54441


In [30]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')
test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.57, Gini: 0.14, KS: 0.12
Eval AUC: 0.54, Gini: 0.09, KS: 0.08
Test AUC: 0.54, Gini: 0.09, KS: 0.07
OOT AUC: 0.54, Gini: 0.09, KS: 0.08


In [31]:
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.4f}, KS: {train_ks:.4f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.4f}, KS: {test_ks:.4f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.4f}, KS: {oot_ks:.4f}")

Train AUC: 0.57, Gini: 0.1411, KS: 0.1154
Test AUC: 0.54, Gini: 0.0876, KS: 0.0736
OOT AUC: 0.54, Gini: 0.0884, KS: 0.0785


In [32]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                             Feature  Importance
2  MEAN(raw.amount WHERE device_os_category = Win...        16.0
1  SUM(raw.amount WHERE action = 1)/SUM(raw.amoun...         6.0
0                                  distinct_merchant         5.0
5                                 max_one_time_spent         4.0
3  NUM_UNIQUE(pg_success_raw_data.month_end_start...         2.0
8                    STD(pg_success_raw_data.amount)         2.0
6                              amt_first_android_txn         2.0
4                      ENTROPY(raw.mode_cleaned)#720         1.0
7                    MAX(pg_success_raw_data.amount)         1.0


In [33]:
feature_importance_df

Feature  Importance
2  MEAN(raw.amount WHERE device_os_category = Win...        16.0
1  SUM(raw.amount WHERE action = 1)/SUM(raw.amoun...         6.0
0                                  distinct_merchant         5.0
5                                 max_one_time_spent         4.0
3  NUM_UNIQUE(pg_success_raw_data.month_end_start...         2.0
8                    STD(pg_success_raw_data.amount)         2.0
6                              amt_first_android_txn         2.0
4                      ENTROPY(raw.mode_cleaned)#720         1.0
7                    MAX(pg_success_raw_data.amount)         1.0

In [34]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
8          0.227011         0.192166        0.198337        0.188385    696   
7          0.172414         0.186924        0.188368        0.184905    696   
6          0.152958         0.182705        0.184843        0.181301    693   
5          0.155523         0.180417        0.181295        0.179791    688   
4          0.170904         0.179317        0.179766        0.179005    708   
3          0.141264         0.178722        0.178989        0.178550    538   
2          0.107492         0.178439        0.178541        0.178089    614   
1          0.118353         0.177784        0.178080        0.177667   2332   

        label_good  label_bad  
Decile                         
8              538        158  
7              576        120  
6              587        106  
5              581        107  
4              587        121  
3              462         76  
2              548         66  
1             2056        276

In [35]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
8          0.200698         0.191611        0.198337        0.187888    573   
7          0.157168         0.186474        0.187836        0.183821    579   
6          0.146010         0.182128        0.183801        0.180776    589   
5          0.130125         0.180240        0.180767        0.179601    561   
4          0.143101         0.179265        0.179596        0.178964    587   
3          0.148325         0.178725        0.178956        0.178550    418   
2          0.130682         0.178439        0.178541        0.178089    528   
1          0.124873         0.177791        0.178080        0.177667   1970   

        label_good  label_bad  
Decile                         
8              458        115  
7              488         91  
6              503         86  
5              488         73  
4              503         84  
3              356         62  
2              459         69  
1             1724        246

In [36]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
8          0.170455         0.193431        0.198337        0.189937    440   
7          0.192683         0.188100        0.189927        0.186869    410   
6          0.138298         0.185205        0.186852        0.182799    470   
5          0.134940         0.181427        0.182737        0.180681    415   
4          0.106430         0.179933        0.180673        0.179427    451   
3          0.163683         0.179091        0.179404        0.178663    391   
2          0.141975         0.178552        0.178614        0.178421    486   
1          0.115269         0.177805        0.178249        0.177667   1336   

        label_good  label_bad  
Decile                         
8              365         75  
7              331         79  
6              405         65  
5              359         56  
4              403         48  
3              327         64  
2              417         69  
1             1182        154

In [37]:
#Save the model
xgb_model.save_model('LeoPayu_PG_only_90_in_24mob.json')
print('Model Saved')

Model Saved
